In [ ]:
!pip install music21

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import string
# Load notes document, extract
# tokens and create sequences

# load notes document into memory
def load_doc(filename):
	file = open(filename, 'r')
	text = file.read()
	file.close()
	return text

# turn the document into clean tokens
def clean_doc(doc):
	# split into tokens by space and make lower case
	tokens = doc.split(' ')
	tokens = [word.lower() for word in tokens]
	return tokens

# save tokens to file, one sequence per line
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

# load document
in_filename = '/content/drive/MyDrive/13.02 colab/huge_notes_offset_augmented_corrected.txt'
doc = load_doc(in_filename)
print(doc[:200])

# clean document
tokens = clean_doc(doc)

# print details
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

"""
# statictics
stats = dict()
for t in tokens:
  if t in stats:
    stats[t] += 1
  else:
    stats[t] = 1
stats = dict(sorted(stats.items(), key=lambda item: item[1]))
file_stats = open('statistics.txt', 'w')
for s in stats:
  file_stats.write(str(stats[s]) + '   ' +  str(s) +'\n')
file_stats.close()
"""
# print first 200 token
print(tokens[:200])

# organize into sequences of tokens
length = 50 + 1 # input:100 target:1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a string line
	line = ' '.join(seq)
	# append
	sequences.append(line)
 
 # print # of total sequences
print('Total Sequences: %d' % len(sequences))

# save sequences to file
out_filename = 'sequences.txt'
save_doc(sequences, out_filename)

67o2 48o52o2 48o52o2 48o52o1 48o53o1 69o71o1 71o1 50o53o1 60o71o1 53o67o2 52o72o1 76o1 52o74o1 72o1 52o71o1 69o1 67o1 50o59o2 50o59o2 50o59o2 60o65o2 65o1 67o1 55o59o1 57o62o1 62o1 64o1 52o55o2 53o59o
Total Tokens: 250798
Unique Tokens: 5353
['67o2', '48o52o2', '48o52o2', '48o52o1', '48o53o1', '69o71o1', '71o1', '50o53o1', '60o71o1', '53o67o2', '52o72o1', '76o1', '52o74o1', '72o1', '52o71o1', '69o1', '67o1', '50o59o2', '50o59o2', '50o59o2', '60o65o2', '65o1', '67o1', '55o59o1', '57o62o1', '62o1', '64o1', '52o55o2', '53o59o2', '53o59o1', '48o52o2', '62o1', '43o59o4', '79o2', '72o84o1', '76o1', '48o1', '76o1', '72o84o1', '76o1', '48o1', '83o1', '77o81o1', '83o86o1', '77o83o1', '72o79o1', '77o1', '79o84o1', '72o88o1', '76o1', '72o86o1', '76o84o1', '83o1', '72o1', '76o81o1', '71o79o1', '74o1', '71o1', '74o1', '71o1', '74o1', '72o76o1', '77o1', '81o1', '77o1', '52o59o1', '67o1', '69o73o1', '74o1', '77o1', '74o1', '52o55o2', '74o76o1', '73o1', '74o77o1', '74o1', '55o59o1', '67o1', '52o72o4',

In [ ]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout, Bidirectional, BatchNormalization
from keras.layers import Embedding
#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from keras.callbacks import History, EarlyStopping, ReduceLROnPlateau
import keras.backend as K

# load document into memory
def load_doc(filename):
	file = open(filename, 'r')
	text = file.read()
	file.close()
	return text

# load
in_filename = 'sequences.txt'
doc = load_doc(in_filename)
# get lines
lines = doc.split('\n')
print(lines[0])

# split train and test sets
test_split = 245000
test_lines = lines[test_split:]
lines = lines[:test_split]
print("train size: ", len(lines))
print("test  size: ", len(test_lines))

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
print(tokenizer.word_index)
sequences = tokenizer.texts_to_sequences(lines)

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

# define model
model = Sequential()
model.add(Embedding(vocab_size, 25, input_length=seq_length))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(LSTM(512))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

# compile model
opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print(K.eval(model.optimizer.lr))

# fit model
history = History()
earlystop = EarlyStopping(monitor='loss', patience= 15, min_delta = 0.1 , verbose=0, mode= 'min')
hist = model.fit(X, y, batch_size=64, epochs=75, callbacks=[earlystop, history])

# save the model and tokenizer to file
model.save('model.h5') 
dump(tokenizer, open('tokenizer.pkl', 'wb'))

# download file
from google.colab import files
files.download('model.h5')

67o2 48o52o2 48o52o2 48o52o1 48o53o1 69o71o1 71o1 50o53o1 60o71o1 53o67o2 52o72o1 76o1 52o74o1 72o1 52o71o1 69o1 67o1 50o59o2 50o59o2 50o59o2 60o65o2 65o1 67o1 55o59o1 57o62o1 62o1 64o1 52o55o2 53o59o2 53o59o1 48o52o2 62o1 43o59o4 79o2 72o84o1 76o1 48o1 76o1 72o84o1 76o1 48o1 83o1 77o81o1 83o86o1 77o83o1 72o79o1 77o1 79o84o1 72o88o1 76o1 72o86o1
train size:  245000
test  size:  5747
{'74o1': 1, '76o1': 2, '72o1': 3, '67o1': 4, '71o1': 5, '69o1': 6, '79o1': 7, '64o1': 8, '77o1': 9, '81o1': 10, '62o1': 11, '60o1': 12, '57o1': 13, '65o1': 14, '55o1': 15, '59o1': 16, '52o1': 17, '83o1': 18, '48o1': 19, '84o1': 20, '50o1': 21, '66o1': 22, '78o1': 23, '68o1': 24, '86o1': 25, '53o1': 26, '52o55o1': 27, '55o59o1': 28, '70o1': 29, '55o59o4': 30, '72o2': 31, '67o2': 32, '48o52o1': 33, '76o2': 34, '64o2': 35, '74o2': 36, '69o2': 37, '55o2': 38, '53o57o1': 39, '62o2': 40, '48o57o1': 41, '75o1': 42, '71o2': 43, '60o2': 44, '73o1': 45, '53o57o4': 46, '52o2': 47, '56o59o1': 48, '50o59o1': 49, '80o1':

In [ ]:
#Save model
with open('model_clsf.json', "w") as json_file:
	json_file.write(model.to_json())
model.save_weights("weights_clsf.h5")
model.summary()
print("Model saved.")

In [ ]:
# plot hyperparams
from matplotlib import pyplot
pyplot.figure(figsize=(8, 3))
pyplot.plot(hist.history['loss'])
pyplot.title('training history')
pyplot.ylabel('value')
pyplot.xlabel('epoch')
pyplot.legend(['Kayıp', 'accuracy'], loc='upper right')
pyplot.show()
pyplot.savefig('history.png')

In [ ]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# load doc into memory
def load_doc(filename):
	file = open(filename, 'r')
	text = file.read()
	file.close()
	return text

# generate a sequence
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

# load cleaned text sequences
in_filename = 'sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

# load model and tokenizer
model = load_model('model.h5')
tokenizer = load(open('tokenizer.pkl', 'rb'))

# select a seed text
seed_text = test_lines[randint(0,len(test_lines))]
seed_elems = seed_text.split(' ')
seed_text = ' '.join(seed_elems[:20])
print(seed_text)

# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 100)
print(generated, '\n')


81o1 72o4 71o76o1 74o1 64o72o1 67o1 72o1 76o1 72o1 76o1 79o1 76o1 79o1 84o1 86o1 84o1 60o82o1 81o1 79o1 77o1


/usr/local/lib/python3.7/dist-packages/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


76o1 74o1 72o4 62o71o4 64o69o1 67o1 65o1 64o1 63o1 64o1 65o1 67o1 69o1 71o1 72o1 71o1 69o1 67o1 65o1 48o64o2 52o76o4 52o72o2 72o74o1 71o1 72o1 50o74o1 76o1 53o77o1 74o1 52o72o2 53o59o1 52o53o1 50o59o1 52o72o1 71o1 53o74o1 77o1 48o76o1 74o1 52o72o2 55o72o2 48o79o2 52o2 55o77o1 76o1 50o74o1 76o1 50o77o4 48o55o3 55o77o1 81o1 55o1 77o1 76o1 74o1 52o72o4 59o76o1 74o1 55o72o2 55o72o2 48o76o2 47o74o1 76o1 74o1 72o1 71o1 72o1 74o1 76o1 77o1 79o1 81o1 83o1 84o1 52o79o4 48o76o4 50o79o1 77o1 60o76o1 77o1 55o79o1 77o1 57o76o1 74o1 60o72o1 67o79o3 67o79o1 81o1 55o72o4 60o79o3 83o1 81o1 79o1 77o1 76o1 74o1 72o1 71o1 69o1 67o1 



In [ ]:
# create midi file from text data
# NOT OFFSET SENSITIVE
import music21

# split words
events = generated.split()
# convert to notes
offset = 0
output_stream = []
for ev in events:
  if 'o' in ev: # if chord
    splitted = [int(i) for i in ev.split('o')]
    chord = music21.chord.Chord(splitted)
    output_stream.append(chord)
  else: # if note
    note = music21.note.Note(int(ev))
    note.offset = offset
    output_stream.append(note)
    offset += 0.5
# write into stream object
s = music21.stream.Stream()
for i in output_stream:
  s.append(i)
print(output_stream)
# write midi file and save
score = music21.stream.Score(output_stream)
score.write('midi', 'generated.mid')

[<music21.chord.Chord B3 D>, <music21.chord.Chord A2 A3 D>, <music21.chord.Chord E3 C4 E>, <music21.chord.Chord D3 B3 D>, <music21.chord.Chord C4 E>, <music21.chord.Chord A2 A3 D>, <music21.chord.Chord B3 D>, <music21.chord.Chord E3 C4 D>, <music21.chord.Chord D3 B3 D>, <music21.chord.Chord C4 E>, <music21.chord.Chord A2 A3 D>, <music21.chord.Chord E3 E>, <music21.chord.Chord E2 D>, <music21.chord.Chord G3 E>, <music21.chord.Chord C3 C4 D>, <music21.chord.Chord C4 E4 E>, <music21.chord.Chord F3 D4 D>, <music21.chord.Chord E4 D>, <music21.chord.Chord D4 D>, <music21.chord.Chord C3 C4 D>, <music21.chord.Chord C3 F-4 F->, <music21.chord.Chord G4 D>, <music21.chord.Chord G3 E4 G#>, <music21.chord.Chord E3 C5 E>, <music21.chord.Chord G3 D5 E>, <music21.chord.Chord G3 E5 E>, <music21.chord.Chord C4 E5 E>, <music21.chord.Chord C5 E->, <music21.chord.Chord C4 D5 D->, <music21.chord.Chord B3 D5 E>, <music21.chord.Chord B3 D5 E>, <music21.chord.Chord D5 D>, <music21.chord.Chord E4 C5 D>, <music2

'000.mid'

In [ ]:
# create midi file from text data
# OFFSET SENSITIVE
import music21

# split words
events = generated.split()
# convert to notes
offset = 0
output_stream = []
for ev in events:
  splitted = [int(i) for i in ev.split('o')]
  # get offset
  q_length = splitted[-1]
  # get notes
  splitted = splitted[:-1]
  if len(splitted) > 1: # if chord
    chord = music21.chord.Chord(splitted, quarterLength = q_length)
    chord.offset = offset
    output_stream.append(chord)
  else: # if note
    note = music21.note.Note(splitted[0], quarterLength = q_length)
    note.offset = offset
    output_stream.append(note)
  # offset
  offset += q_length
s = music21.stream.Stream()
# write into stream object
for i in output_stream:
  s.append(i)
print(output_stream)
# write midi file and save
score = music21.stream.Score(output_stream)
score.write('midi', '000.mid')

[<music21.note.Note E>, <music21.note.Note D>, <music21.note.Note C>, <music21.chord.Chord D4 B4>, <music21.chord.Chord E4 A4>, <music21.note.Note G>, <music21.note.Note F>, <music21.note.Note E>, <music21.note.Note E->, <music21.note.Note E>, <music21.note.Note F>, <music21.note.Note G>, <music21.note.Note A>, <music21.note.Note B>, <music21.note.Note C>, <music21.note.Note B>, <music21.note.Note A>, <music21.note.Note G>, <music21.note.Note F>, <music21.chord.Chord C3 F-4>, <music21.chord.Chord E3 E5>, <music21.chord.Chord E3 C5>, <music21.chord.Chord C5 D5>, <music21.note.Note B>, <music21.note.Note C>, <music21.chord.Chord D3 D5>, <music21.note.Note E>, <music21.chord.Chord F3 F5>, <music21.note.Note D>, <music21.chord.Chord E3 C5>, <music21.chord.Chord F3 B3>, <music21.chord.Chord E3 F3>, <music21.chord.Chord D3 B3>, <music21.chord.Chord E3 C5>, <music21.note.Note B>, <music21.chord.Chord F3 D5>, <music21.note.Note F>, <music21.chord.Chord C3 E5>, <music21.note.Note D>, <music21.c

'000.mid'

In [ ]:
# create midi file from seed data
# OFFSET SENSITIVE
import music21

# split words
events = seed_text.split()
# convert to notes
offset = 0
output_stream = []
for ev in events:
  splitted = [int(i) for i in ev.split('o')]
  # get offset
  q_length = splitted[-1]
  # get notes
  splitted = splitted[:-1]
  if len(splitted) > 1: # if chord
    chord = music21.chord.Chord(splitted, quarterLength = q_length)
    chord.offset = offset
    output_stream.append(chord)
  else: # if note
    note = music21.note.Note(splitted[0], quarterLength = q_length)
    note.offset = offset
    output_stream.append(note)
  # offset
  offset += q_length
s = music21.stream.Stream()
# write into stream object
for i in output_stream:
  s.append(i)
print(output_stream)
# write midi file and save
score = music21.stream.Score(output_stream)
score.write('midi', 'seed.mid')

[<music21.note.Note A>, <music21.note.Note C>, <music21.chord.Chord B4 E5>, <music21.note.Note D>, <music21.chord.Chord E4 C5>, <music21.note.Note G>, <music21.note.Note C>, <music21.note.Note E>, <music21.note.Note C>, <music21.note.Note E>, <music21.note.Note G>, <music21.note.Note E>, <music21.note.Note G>, <music21.note.Note C>, <music21.note.Note D>, <music21.note.Note C>, <music21.chord.Chord C4 B-5>, <music21.note.Note A>, <music21.note.Note G>, <music21.note.Note F>]


'seed.mid'

In [ ]:
# ***NOTES*** #

"""
# statictics
stats = dict()
for t in tokens:
  if t in stats:
    stats[t] += 1
  else:
    stats[t] = 1
stats = dict(sorted(stats.items(), key=lambda item: item[1]))
file_stats = open('statistics.txt', 'w')
for s in stats:
  file_stats.write(str(stats[s]) + '   ' +  str(s) +'\n')
file_stats.close()
"""